In [3]:
### Tools used for clean up
import numpy as np
import pandas as pd
import os

In [4]:
### Global
Burn = int(input("What Burn would you like to clean? (1-6):"))
fill_nan = np.nan

change_nan = input("Would you like to change the NaN's to a different value? (y/n):")
if change_nan == "y":
    fill_nan = input("What to replace NaN's with? ex: 9999:")

if Burn == 1: 
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_first_set_truss_burns/SERDP%20burn%20one/"
if Burn == 2:
    path ="http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_first_set_truss_burns/SERDP%20burn%20two/"
if Burn == 3:
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_first_set_truss_burns/SERDP%20burn%20three/"
if Burn == 4:
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_second_set_truss_burns/SLEF%20second%20set%20truss%20burns/SERDP%20burn%20four/"
if Burn == 5:
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_second_set_truss_burns/SLEF%20second%20set%20truss%20burns/SERDP%20burn%20five/"
if Burn  == 6:
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_second_set_truss_burns/SLEF%20second%20set%20truss%20burns/SERDP%20burn%20six/"
if Burn ==7:
    path = 'http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_third_set_truss_burns/SERDP_Burn_Seven/'
if Burn ==8:
    path = "http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_third_set_truss_burns/SERDP_Burn_Eight/"
    

What Burn would you like to clean? (1-6):7
Would you like to change the NaN's to a different value? (y/n):n


In [5]:
def file_to_df(path, file, Burn):
   
        
    df = pd.read_csv(path+file, skiprows=4, na_values='NAN')
    headers_df = pd.read_csv(path+file,skiprows=1,nrows = 0)
    
    df.columns=list(headers_df) 
    df.fillna(value=fill_nan, inplace=True)
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    return(df)

In [6]:
### Finding where to cut the data, needs all df, gives time_start and time_end 
### needed for truncater()

def timestamp_matcher(df_names, file_num):
    
    
    min_lst, max_lst =[],[]

    for i in df_names:
        min_lst.append(i["TIMESTAMP"].min())
        max_lst.append(i["TIMESTAMP"].max())

        fmt = "File: {} | Start: {} | End: {}" 
    for i in range(len(min_lst)):
        print(fmt.format(file_num[i], min_lst[i],max_lst[i] ))

    time_start, time_end = max(min_lst), min(max_lst)    
    print()
    print("Start timestamp Pulled:",time_start, "| End Timestamp Pulled:",time_end)

    finder = ['5T', '30S','S','.1S']
    for n in range(len(finder)):
        test_start=list(pd.date_range(start=time_start, end=time_end,freq = finder[n]))
        test_end = test_start[::-1]
        time=[]
        for t in range(len(test_start)): 
            for df in range(len(df_names)):
                if test_start[t] not in list(df_names[df]["TIMESTAMP"]):
                    break
                else:
                    time.append(test_start[t])
                    break
            if len(time)==1:
                if t ==0 or n == len(finder)-1:
                    time_start = test_start[t]
                else:
                    time_start = test_start[t-1]
                break


        time=[]
        for t in range(len(test_end)):
            for df in range(len(df_names)):
                if test_end[t] not in list(df_names[df]["TIMESTAMP"]):
                    break
                else:
                    time.append(test_end[t])
                    break
            if len(time)==1:
                if t ==0 or n == len(finder)-1:
                    time_end = test_end[t]
                else:
                    time_end = test_end[t-1]
                break
    print()
    print("Timestamp that can actually be used to trim due to gaps:")
    print("Start Time:", str(time_start), "| End Time:", str(time_end))
    
    return time_start, time_end


In [7]:
def cutter(df, time_start, time_end):

    s_index = df.index[df["TIMESTAMP"]==time_start].tolist()[-1]
    e_index = df.index[df["TIMESTAMP"]==time_end].tolist()[0]
    df = df.truncate(before=s_index, after= e_index)
    df=df.reset_index(drop = True)
    return df

In [8]:
def repeat(df):
    #### Check for repeated times

    lst=list(df["TIMESTAMP"])
    u = list(np.unique(lst)) 
    if len(u) != len(df):
        repeat_1, repeat_2, repeat_index = [],[],[]
        for i in u:
            x = lst.count(i)
            if x>1:
                y=[j for j,val in enumerate(lst) if val==i]
                repeat_index.append(y)
                repeat_1.append(list(df.loc[y[0],:]))
                repeat_2.append(list(df.loc[y[1],:]))
        if len(repeat_index)>0:
            print("Yikes! Number of repeated times: ",len(repeat_index),)
            print("Start:", repeat_index[0],"End:",repeat_index[-1])
            print("Time stamp repeats:",repeat_1[0][0],"-", repeat_1[-1][0], "Length", len(repeat_1))
            return repeat_2[-1][0]
    else:
        print("Hurray! No time Repeats")

In [9]:
def time_columns(df):
    year_lst  = list(np.full(len(df),fill_nan))
    month_lst = list(np.full(len(df),fill_nan))
    day_lst   = list(np.full(len(df),fill_nan))
    hour_lst  = list(np.full(len(df),fill_nan))
    min_lst   = list(np.full(len(df),fill_nan))
    second_lst= list(np.full(len(df),fill_nan))

    ### Parcing the timestamps and seperating them 
    time_stmp_lst = list(df["TIMESTAMP"].astype(str))
    for i in range(len(df)):
        time_step    = time_stmp_lst[i].replace("-", ",").replace(":",",").replace(" ",",").replace("/",',').split(",")
        year_lst[i]  = "{:.0f}".format(float(time_step[0])).zfill(4)
        month_lst[i] = "{:.0f}".format(float(time_step[1])).zfill(2)
        day_lst[i]   = "{:.0f}".format(float(time_step[2])).zfill(2)
        hour_lst[i]  = "{:.0f}".format(float(time_step[3])).zfill(2)
        min_lst[i]   = "{:.0f}".format(float(time_step[4])).zfill(2)
        second_lst[i]= "{:.1f}".format(float(time_step[5])).zfill(4)
    
    df_time = pd.DataFrame()
    df_time["YYYY"] = year_lst
    df_time["MM"]   = month_lst
    df_time["DD"]   = day_lst
    df_time["Hour"] = hour_lst
    df_time["Min"]  = min_lst
    df_time["Sec"]  = second_lst
    
    return df_time

In [13]:
### Global variables 
files = ["TOA5_2878.WGNover10Hz.dat","TOA5_2879.ts_data.dat", "TOA5_4390.ts_data.dat",\
         "TOA5_4975.ts_data.dat","TOA5_4976.ts_data.dat","TOA5_10442.ts_data.dat","TOA5_11585.ts_data.dat"]
files =["TOA5_2878.WGNover10Hz.dat","TOA5_2879.ts_data.dat",\
             "TOA5_3884.ts_data.dat", "TOA5_4390.ts_data.dat",\
             "TOA5_4975.ts_data.dat","TOA5_4976.ts_data.dat",
             "TOA5_10442.ts_data.dat", "TOA5_11584.ts_data.dat",\
             "TOA5_11585.ts_data.dat"]
files = ["TOA5_2879.ts_data.dat",\
             "TOA5_3884.ts_data.dat", "TOA5_4390.ts_data.dat",\
             "TOA5_4975.ts_data.dat","TOA5_4976.ts_data.dat",
             "TOA5_10442.ts_data.dat", "TOA5_11584.ts_data.dat",\
             "TOA5_11585.ts_data.dat"]

#df_2878 = file_to_df(path, files[0], Burn)### Don't use on burns 1 and 2, tower_data
df_2879 = file_to_df(path, files[0], Burn)
df_3884 = file_to_df(path, files[1], Burn)
df_4390 = file_to_df(path, files[2], Burn)
df_4975 = file_to_df(path, files[3], Burn)

df_4976 = file_to_df(path, files[4], Burn)
df_10442 = file_to_df(path, files[5], Burn)

df_11584 = file_to_df(path, files[6], Burn)
df_11585 = file_to_df(path, files[7], Burn)

#d= file_to_df(path,files[7],Burn)
#r= file_to_df(path,files[8], Burn)

In [26]:
df_names= [df_2879, df_3884, df_4390, df_4975, df_4976, df_10442,\
               df_11584, df_11585]
file_num = ["2879","3884","4390","4975","4976",\
                "10442",'11584',"11585"]
t_s,t_e = timestamp_matcher(df_names,file_num)
t_s='2018-05-09 10:15:38.4'
trim_df = input("Would you like to trim the data to these timestamps? (y/n):")
if trim_df.lower() == "y":
        df_2879 = cutter(df_2879, t_s, t_e)
        df_3884 = cutter(df_3884, t_s, t_e)
        df_4390 = cutter(df_4390, t_s, t_e)
        df_4975 = cutter(df_4975, t_s, t_e)
        df_4976 = cutter(df_4976, t_s, t_e)
        df_10442 = cutter(df_10442, t_s, t_e)
        df_11584 = cutter(df_11584, t_s, t_e)
        df_11585 = cutter(df_11585, t_s, t_e)

File: 2879 | Start: 2018-05-09 10:12:04.200000 | End: 2018-05-09 11:06:56.100000
File: 3884 | Start: 2018-05-09 10:12:04.200000 | End: 2018-05-09 11:06:56.100000
File: 4390 | Start: 2018-05-09 10:12:04.200000 | End: 2018-05-09 11:06:56.100000
File: 4975 | Start: 2018-05-09 10:12:04.200000 | End: 2018-05-09 11:06:56.100000
File: 4976 | Start: 2018-03-17 15:15:24.800000 | End: 2018-05-09 11:11:44.200000
File: 10442 | Start: 2018-03-17 14:40:50.900000 | End: 2018-05-09 11:14:15.200000
File: 11584 | Start: 2018-03-04 10:24:28 | End: 2018-05-09 11:11:52.200000
File: 11585 | Start: 2018-03-17 14:42:01.800000 | End: 2018-05-09 11:12:42.800000

Start timestamp Pulled: 2018-05-09 10:12:04.200000 | End Timestamp Pulled: 2018-05-09 11:06:56.100000

Timestamp that can actually be used to trim due to gaps:
Start Time: 2018-05-09 10:12:04.200000 | End Time: 2018-05-09 11:02:04.200000
Would you like to trim the data to these timestamps? (y/n):y


In [21]:
df_names= [df_2879,df_4390,df_4975,df_4976,df_10442,df_11585]
file_num = ["2879","4390","4975","4390","4975","4976","10442","11585"]
t_s,t_e = timestamp_matcher(df_names,file_num)
t_s="2018-05-09 08:26:56.1"
trim_df = input("Would you like to trim the data to these timestamps? (y/n):")
if trim_df.lower() == "y":
    #df_2878 = cutter(df_2878, t_s, t_e)
    df_2879 = cutter(df_2879, t_s, t_e)
    df_4390 = cutter(df_4390, t_s, t_e)
    df_4975 = cutter(df_4975, t_s, t_e)
    df_4976 = cutter(df_4976, t_s, t_e)
    df_10442 = cutter(df_10442, t_s, t_e)
    df_11585 = cutter(df_11585, t_s, t_e)

File: 2879 | Start: 2018-03-17 14:36:23.900000 | End: 2018-05-09 11:14:01.300000
File: 4390 | Start: 2018-03-17 14:45:46.600000 | End: 2018-05-09 11:12:56.700000
File: 4975 | Start: 2018-03-17 15:08:08.400000 | End: 2018-05-09 11:14:53.900000
File: 4390 | Start: 2018-03-17 15:15:24.800000 | End: 2018-05-09 11:11:44.200000
File: 4975 | Start: 2018-03-17 14:40:50.900000 | End: 2018-05-09 11:14:15.200000
File: 4976 | Start: 2018-03-17 14:42:01.800000 | End: 2018-05-09 11:12:42.800000

Start timestamp Pulled: 2018-03-17 15:15:24.800000 | End Timestamp Pulled: 2018-05-09 11:11:44.200000


KeyboardInterrupt: 

In [ ]:
df_names= [df_2878,df_2879,df_4390,df_4975,df_4976,df_10442,df_11585]
file_num = ["2878","2879","4390","4975","4390","4975","4976","10442","11585"]
#### use the t_s and t_e from above
#t_s,t_e = timestamp_matcher(df_names)

check = input("Would you like to check for repeated timestamps? Note: if there are repeats, it takes at least 10 mins if there are repeats (y/n):")

if check == "y":
    end_repeat_times = []
    for i in range(len(df_names)):
        print(file_num[i]+':')
        end_repeat_times.append(repeat(df_names[i])) 
cut_out_repeat = "n"
for i in end_repeat_times:
    if str(i) != "None":
        cut_out_repeat = input("Would you like to cut all data at the end of the repeated times? (y/n):")
        t_s = i
if cut_out_repeat == "y":     
    #df_2878 = cutter(df_2878, t_s, t_e)
    df_2879 = cutter(df_2879, t_s, t_e)
    df_4390 = cutter(df_4390, t_s, t_e)
    df_4975 = cutter(df_4975, t_s, t_e)
    df_4976 = cutter(df_4976, t_s, t_e)
    df_10442 = cutter(df_10442, t_s, t_e)
    df_11585 = cutter(df_11585, t_s, t_e)

In [ ]:
### Grabbing Sonic data from specific files
sonic_columns=["Ux_","Uy_","Uz_","Ts_"]
time_columns_lst=["YYYY","MM","DD","Hour","Min","Sec"]

df_A1, df_A2, df_A3, df_A4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
df_B1, df_B2, df_B3, df_B4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
df_C1, df_C2, df_C3, df_C4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
df_D1, df_D2, df_D3, df_D4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()

a_row_lst = [df_A1, df_A2, df_A3, df_A4]
b_row_lst = [df_B1, df_B2, df_B3, df_B4]
c_row_lst = [df_C1, df_C2, df_C3, df_C4]
d_row_lst = [df_D1, df_D2, df_D3, df_D4 ]

all_sonics = a_row_lst+ b_row_lst + c_row_lst +d_row_lst

###  Burns 1-6 truss:
df_4975_time, df_2879_time = time_columns(df_4975), time_columns(df_2879)
df_11585_time, df_4976_time = time_columns(df_11585), time_columns(df_4976)
### WG Nover 10hz
df_2878_time = time_columns(df_2878)
df_WGNover = pd.DataFrame()

for col in sonic_columns:
    df_WGNover[col+"1"] = df_2878[col+"1"]
for t in time_columns_lst:
    df_WGNover[t] = df_2878_time[t]

    
for n in range(len(a_row_lst)):
    
    for i in range(len(sonic_columns)):
        a_row_lst[n][sonic_columns[i]+str(n+1)] = df_4975[sonic_columns[i]+str(n+1)]
        b_row_lst[n][sonic_columns[i]+str(n+1)] = df_2879[sonic_columns[i]+str(n+1)]
        c_row_lst[n][sonic_columns[i]+str(n+1)] = df_11585[sonic_columns[i]+str(n+1)]
        d_row_lst[n][sonic_columns[i]+str(n+1)] = df_4976[sonic_columns[i]+str(n+1)]
    
    for i in range(len(time_columns_lst)):
        a_row_lst[n][time_columns_lst[i]]=df_4975_time[time_columns_lst[i]]
        b_row_lst[n][time_columns_lst[i]]=df_2879_time[time_columns_lst[i]]
        c_row_lst[n][time_columns_lst[i]]=df_11585_time[time_columns_lst[i]]
        d_row_lst[n][time_columns_lst[i]]=df_4976_time[time_columns_lst[i]]

In [ ]:
#### Thermal Couple data
time_columns_lst=["YYYY","MM","DD","Hour","Min","Sec"]

df_B1_tc, df_B2_tc, df_B3_tc, df_B4_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
df_C1_tc, df_C2_tc, df_C3_tc, df_C4_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()

first_tc_group = [ df_B1_tc, df_B3_tc, df_C1_tc, df_C3_tc]
secnd_tc_group = [ df_B2_tc, df_B4_tc, df_C2_tc, df_C4_tc]

df_10442_time, df_2879_time = time_columns(df_10442), time_columns(df_2879)
df_4390_time, df_11585_time = time_columns(df_4390), time_columns(df_11585)

t_c_lst_1 = ["Temp_C(1)","Temp_C(2)","Temp_C(3)","Temp_C(4)","Temp_C(5)","Temp_C(6)","Temp_C(7)"]
t_c_lst_2 = ["Temp_C(8)","Temp_C(9)","Temp_C(10)","Temp_C(11)","Temp_C(12)","Temp_C(13)","Temp_C(14)"]

df_tc_lst= [df_10442, df_2879, df_4390, df_11585]
df_time_lst = [df_10442_time, df_2879_time, df_4390_time, df_11585_time] 

for j in range(len(first_tc_group)):
    for i in range(len(t_c_lst_1)):
        first_tc_group[j][t_c_lst_1[i]]= df_tc_lst[j][t_c_lst_1[i]]
        secnd_tc_group[j][t_c_lst_2[i]]= df_tc_lst[j][t_c_lst_2[i]]
    
    for t in range(len(time_columns_lst)):
        first_tc_group[j][time_columns_lst[t]]= df_time_lst[j][time_columns_lst[t]]
        secnd_tc_group[j][time_columns_lst[t]]= df_time_lst[j][time_columns_lst[t]]

In [ ]:
### Creating the directories to save the data
cwd = os.getcwd()
save_dir = "Burn-"+str(Burn)
os.mkdir(cwd+"/" + save_dir)

tc_dir =cwd+"/" + save_dir + "/thermal_couples"
sonic_dir =cwd+"/" + save_dir + "/sonics"

os.mkdir(tc_dir)
os.mkdir(sonic_dir)

In [ ]:
### Save the Sonic data
save_as_lst = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4", "C1","C2","C3","C4","D1","D2","D3","D4"]
for i in range(len(all_sonics)):
    all_sonics[i].to_csv(sonic_dir+'/'+save_as_lst[i]+"_UVWT_Burn-"+str(Burn)+".txt", sep='\t',index=False)
df_WGNover.to_csv(sonic_dir+'/WGNover_UVWT_Burn-'+str(Burn)+".txt",sep='\t',index=False)

In [ ]:
### Saving Thermal Couple dataframes
all_tc_group = first_tc_group + secnd_tc_group
save_as_lst = ["B1", "B2", "B3", "B4", "C1","C2","C3","C4"]
for i in range(len(all_tc_group)):
    all_tc_group[i] = all_tc_group[i].round(3)
    all_tc_group[i].to_csv(tc_dir+'/'+save_as_lst[i]+"_thermal_couple_Burn-"+str(Burn)+".txt", sep='\t',index=False)

In [ ]:
####### BURN 7

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  2 18:18:26 2019

@author: joey
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul  1 14:33:38 2019

@author: joey
"""



### Global
#Burn = int(input("What Burn would you like to clean? (7 or 8):"))

path = 'http://35.12.130.8/study/2018-SERDP/02-raw-data/ALL-RAW/SERDP_SLEF_Burn_data/SERDP_SLEF_Sonic/SLEF_third_set_truss_burns/SERDP_Burn_Seven/'

Burn = 7
def compiler_b7():
    files = ["TOA5_2879.ts_data.dat",\
             "TOA5_3884.ts_data.dat", "TOA5_4390.ts_data.dat",\
             "TOA5_4975.ts_data.dat","TOA5_4976.ts_data.dat",
             "TOA5_10442.ts_data.dat", "TOA5_11584.ts_data.dat",\
             "TOA5_11585.ts_data.dat"]

    file_num = ["2879","3884","4390","4975","4976",\
                "10442",'11584',"11585"]
    ### First Loading the files into the script
    #df_2878 = file_to_df(path, files[0])
    
    df_2879 = file_to_df(path, files[0],Burn)
    df_3884 = file_to_df(path, files[1],Burn)
    df_4390 = file_to_df(path, files[2], Burn)
    df_4975 = file_to_df(path, files[3], Burn)
    
    df_4976 = file_to_df(path, files[4],Burn)
    df_10442 = file_to_df(path, files[5], Burn)
    
    df_11584 = file_to_df(path, files[6],Burn)
    df_11585 = file_to_df(path, files[7],Burn)
    ### List of dataframes that is needed
    df_names= [df_2879, df_3884, df_4390, df_4975, df_4976, df_10442,\
               df_11584, df_11585]

    t_s,t_e = timestamp_matcher(df_names,file_num)
    trim_df = input("Would you like to trim the data to these timestamps? (y/n):")
    if trim_df.lower() == "y":
        df_2879 = cutter(df_2879, t_s, t_e)
        df_3884 = cutter(df_3884, t_s, t_e)
        df_4390 = cutter(df_4390, t_s, t_e)
        df_4975 = cutter(df_4975, t_s, t_e)
       # df_4976 = cutter(df_4976, t_s, t_e)
        df_10442 = cutter(df_10442, t_s, t_e)
        df_11584 = cutter(df_11584, t_s, t_e)
        df_11585 = cutter(df_11585, t_s, t_e)

    df_names= [ df_2879, df_3884, df_4390, df_4975, df_4976, df_10442,\
               df_11584, df_11585]
    check = input("Would you like to check for repeated timestamps? Note: if there are repeats, it takes at least 10 mins if there are repeats (y/n):")

    if check == "y":
        end_repeat_times = []
        for i in range(len(df_names)):
            print(file_num[i]+':')
            end_repeat_times.append(repeat(df_names[i])) 
    cut_out_repeat = "n"
    for i in end_repeat_times:
        if str(i) != "None":
            cut_out_repeat = input("Would you like to cut all data at the end of the repeated times? (y/n):")
            t_s = i
    if cut_out_repeat == "y":     
        df_2879 = cutter(df_2879, t_s, t_e)
        df_3884 = cutter(df_3884, t_s, t_e)
        df_4390 = cutter(df_4390, t_s, t_e)
        df_4975 = cutter(df_4975, t_s, t_e)
        df_4976 = cutter(df_4976, t_s, t_e)
        df_10442 = cutter(df_10442, t_s, t_e)
        df_11584 = cutter(df_11584, t_s, t_e)
        df_11585 = cutter(df_11585, t_s, t_e)

    ### Grabbing Sonic data from specific files
    sonic_columns=["Ux_","Uy_","Uz_","Ts_"]
    time_columns_lst=["YYYY","MM","DD","Hour","Min","Sec"]
    
    df_A1, df_A2, df_A3, df_A4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    df_B1, df_B2, df_B3, df_B4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    df_C1, df_C2, df_C3, df_C4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    df_D1, df_D2, df_D3, df_D4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    
    a_row_lst = [df_A1, df_A2, df_A3, df_A4]
    b_row_lst = [df_B1, df_B2, df_B3, df_B4]
    c_row_lst = [df_C1, df_C2, df_C3, df_C4]
    d_row_lst = [df_D1, df_D2, df_D3, df_D4 ]
    
    all_sonics = a_row_lst+ b_row_lst + c_row_lst +d_row_lst
    
    ###  Burns 7/8 truss:
    df_4975_time, df_2879_time = time_columns(df_4975), time_columns(df_2879)
    df_11585_time, df_4976_time = time_columns(df_11585), time_columns(df_4976)
    df_3884_time, df_11584_time = time_columns(df_3884), time_columns(df_11584)
    
    df_10442_time = time_columns(df_10442)
    df_4390_time = time_columns(df_4390)
    
    ### WG Nover 10hz
    #df_2878_time = time_columns(df_2878)
    #df_WGNover = pd.DataFrame()
    
    #for col in sonic_columns:
     #   df_WGNover[col+"1"] = df_2878[col+"1"]
    #for t in time_columns_lst:
    #    df_WGNover[t] = df_2878_time[t]
    
        
    for n in range(len(a_row_lst)):
        
        for i in range(len(sonic_columns)):
            a_row_lst[n][sonic_columns[i]+str(n+1)] = df_4975[sonic_columns[i]+str(n+1)]
            b_row_lst[n][sonic_columns[i]+str(n+1)] = df_10442[sonic_columns[i]+str(n+1)]
            c_row_lst[n][sonic_columns[i]+str(n+1)] = df_11584[sonic_columns[i]+str(n+1)]
            d_row_lst[n][sonic_columns[i]+str(n+1)] = df_4390[sonic_columns[i]+str(n+1)]
        
        for i in range(len(time_columns_lst)):
            a_row_lst[n][time_columns_lst[i]]=df_4975_time[time_columns_lst[i]]
            b_row_lst[n][time_columns_lst[i]]=df_10442_time[time_columns_lst[i]]
            c_row_lst[n][time_columns_lst[i]]=df_11584_time[time_columns_lst[i]]
            d_row_lst[n][time_columns_lst[i]]=df_4390_time[time_columns_lst[i]]

    #### Thermal Couple data
    time_columns_lst=["YYYY","MM","DD","Hour","Min","Sec"]
    
    df_B1_tc, df_B2_tc, df_B3_tc, df_B4_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    df_C1_tc, df_C2_tc, df_C3_tc, df_C4_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    
    df_B5_tc, df_B6_tc, df_B7_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    df_C5_tc, df_C6_tc, df_C7_tc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
    
    t_c_lst_1 = ["Temp_C(1)","Temp_C(2)","Temp_C(3)","Temp_C(4)","Temp_C(5)","Temp_C(6)","Temp_C(7)"]
    t_c_lst_2 = ["Temp_C(8)","Temp_C(9)","Temp_C(10)","Temp_C(11)","Temp_C(12)","Temp_C(13)","Temp_C(14)"]
    
    first_tc_group = [ df_B2_tc, df_B3_tc, df_B6_tc, df_B7_tc,\
                       df_C2_tc, df_C3_tc, df_C6_tc, df_C7_tc]
    
    secnd_tc_group = [ df_B1_tc, df_B4_tc, df_B5_tc,\
                      df_C1_tc, df_C4_tc, df_C5_tc]

    df_tc_lst_1 = [df_4975, df_2879, df_3884, df_10442, df_11585, df_4976,\
                df_4390, df_11584]
    
    df_time_lst_1 =[df_4975_time, df_2879_time, df_3884_time, df_10442_time,\
                    df_11585_time, df_4976_time, df_4390_time, df_11584_time]
    for j in range(len(first_tc_group)):
        for i in range(len(t_c_lst_1)):
            first_tc_group[j][t_c_lst_1[i]]= df_tc_lst_1[j][t_c_lst_1[i]]
           
        for t in range(len(time_columns_lst)):
            first_tc_group[j][time_columns_lst[t]]= df_time_lst_1[j][time_columns_lst[t]]
    
    df_tc_lst_2 = [df_2879, df_4975, df_10442, df_4976, df_11585, df_11584]
    
    df_time_lst_2 = [df_2879_time, df_4975_time, df_10442_time, df_4976_time,\
                   df_11585_time, df_11584_time]
    
    for j in range(len(secnd_tc_group)):
        for i in range(len(t_c_lst_2)):
            secnd_tc_group[j][t_c_lst_2[i]]= df_tc_lst_2[j][t_c_lst_2[i]]
           
        for t in range(len(time_columns_lst)):
            secnd_tc_group[j][time_columns_lst[t]]= df_time_lst_2[j][time_columns_lst[t]]
   
    all_tc_group = [df_B1_tc, df_B2_tc, df_B3_tc, df_B4_tc, df_B5_tc,\
                    df_B6_tc, df_B7_tc,df_C1_tc, df_C2_tc, df_C3_tc, df_C4_tc,\
                    df_C5_tc, df_C6_tc, df_C7_tc]
    
    return all_sonics, all_tc_group #, df_WGNover

def saver_b7():
    
    all_sonics, all_tc_group = compiler_b7()
    
    ### Creating the directories to save the data
    save_me = input("Would you like to save the data into the working directory? (y/n):")
    if save_me == "y":
        cwd = os.getcwd()
    else:
        cwd = input("Full path of save directory:")
        
    save_dir = "Burn-"+str(Burn)
    os.mkdir(cwd+"/" + save_dir)
    
    tc_dir =cwd+"/" + save_dir + "/thermal_couples"
    sonic_dir =cwd+"/" + save_dir + "/sonics"
    
    os.mkdir(tc_dir)
    os.mkdir(sonic_dir)

    ### Save the Sonic data
    save_as_lst = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4", "C1","C2",\
                   "C3","C4","D1","D2","D3","D4"]
    for i in range(len(all_sonics)):
        sv_file = sonic_dir+'/'+save_as_lst[i]+"_UVWT_Burn-"+str(Burn)+".txt"
        all_sonics[i].to_csv(sv_file, sep='\t',index=False)
    #df_WGNover.to_csv(sonic_dir+'/WGNover_UVWT_Burn-'+str(Burn)+".txt",sep='\t',index=False)

    ### Saving Thermal Couple dataframes
    
    save_as_lst = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "C1","C2","C3",\
                   "C4", "C5", "C6", "C7"]
    
    for i in range(len(all_tc_group)):
        all_tc_group[i] = all_tc_group[i].round(3)
        sv_file=tc_dir+'/'+save_as_lst[i]+"_thermal_couple_Burn-"+str(Burn)+".txt"
        all_tc_group[i].to_csv(sv_file, sep='\t',index=False)
    print("You now have the Burn sonics and thermocouple saved")



saver_b7()


File: 2879 | Start: 2018-03-17 14:36:23.900000 | End: 2018-05-09 11:14:01.300000
File: 3884 | Start: 2018-05-08 17:57:11.400000 | End: 2018-05-09 11:15:12.100000
File: 4390 | Start: 2018-03-17 14:45:46.600000 | End: 2018-05-09 11:12:56.700000
File: 4975 | Start: 2018-03-17 15:08:08.400000 | End: 2018-05-09 11:14:53.900000
File: 4976 | Start: 2018-03-17 15:15:24.800000 | End: 2018-05-09 11:11:44.200000
File: 10442 | Start: 2018-03-17 14:40:50.900000 | End: 2018-05-09 11:14:15.200000
File: 11584 | Start: 2018-03-04 10:24:28 | End: 2018-05-09 11:11:52.200000
File: 11585 | Start: 2018-03-17 14:42:01.800000 | End: 2018-05-09 11:12:42.800000

Start timestamp Pulled: 2018-05-08 17:57:11.400000 | End Timestamp Pulled: 2018-05-09 11:11:44.200000

Timestamp that can actually be used to trim due to gaps:
Start Time: 2018-05-09 08:23:09.900000 | End Time: 2018-05-09 11:07:11.400000
Would you like to trim the data to these timestamps? (y/n):y


IndexError: list index out of range

In [1]:
l=[1]

In [2]:
l[-1]

1